    papermill -p PDFPATH /home/aubrey/ALBI-345/files/pdfs/Orders-with-Complete-Metamorphosis.pdf pdf2reveal.ipynb out.ipynb

In [1]:
import os
import requests, zipfile, io
import jinja2
import glob
import shutil

* https://marcus-baw.medium.com/using-reveal-js-a74b30e4065b
* https://martinomensio.medium.com/how-to-host-reveal-js-slides-on-github-pages-and-have-a-tidy-repository-1a363944c38d
* https://superuser.com/questions/1178666/imagemagick-convert-quits-after-some-pages

In [2]:
# Parameters which may be set using papermill

PDFPATH = '/home/aubrey/Downloads/applied-ent.pdf'

In [3]:
# Parameters
PDFPATH = "/home/aubrey/ALBI-345/files/pdfs/Orders-with-Complete-Metamorphosis.pdf"


In [4]:
def install_reveal():
    """
    Download and install the latest version of reveal.js.
    """
    if not os.path.exists('reveal.js-master'):
        r = requests.get('https://github.com/hakimel/reveal.js/archive/master.zip')
        z = zipfile.ZipFile(io.BytesIO(r.content))
        z.extractall()

In [5]:
def create_template():
    """
    Create a jinja2 template by editing the reveal.js default index.html
    """

    # Read the default index.html as a string    
    s = open('reveal.js-master/index.html', 'r').read()

    # edit the string and save it as template.html
    s = s.replace('"dist', '"../reveal.js-master/dist')
    s = s.replace('"plugin', '"../reveal.js-master/plugin')
    s = s.replace('<section>Slide 1</section>\n\t\t\t\t<section>Slide 2</section>',
              '{% for image in images %}\n\t\t\t\t\t<section> <img src="../{{ image }}"> </section>\n\t\t\t\t{% endfor %}')

    with open('template.html', 'w') as f:
        f.write(s)

In [6]:
def create_presentation():
    """
    Use the jinja2 template to add slides from the images directory
    """
    
    if not os.path.exists('images'):
        os.mkdir('images')    
    
    # Create a directory for the presentation and put a copy of the PDF in it
    pdf_name = os.path.basename(PDFPATH).replace('.pdf', '')
    os.mkdir(pdf_name)
    shutil.copyfile(PDFPATH, f'{pdf_name}/{pdf_name}.pdf')
    
    # Convert the PDF pages into jpg images and store them in the communal images directory
    os.system(f'convert -density 300 {PDFPATH} images/{pdf_name}-%03d.jpg')
    images = sorted(glob.glob(f'images/{pdf_name}-???.jpg'))  
    print(images)
       
    templateLoader = jinja2.FileSystemLoader(searchpath="./")
    templateEnv = jinja2.Environment(loader=templateLoader)
    TEMPLATE_FILE = "template.html"
    template = templateEnv.get_template(TEMPLATE_FILE)
    outputText = template.render({'images': images})

    with open(f'{pdf_name}/index.html', 'w') as out:
        out.write(outputText)

In [7]:
# MAIN

if not os.path.exists('reveal.js-master'):
    install_reveal()
    create_template()
# create_presentation()

In [8]:
create_presentation()

['images/Orders-with-Complete-Metamorphosis-000.jpg', 'images/Orders-with-Complete-Metamorphosis-001.jpg', 'images/Orders-with-Complete-Metamorphosis-002.jpg', 'images/Orders-with-Complete-Metamorphosis-003.jpg', 'images/Orders-with-Complete-Metamorphosis-004.jpg', 'images/Orders-with-Complete-Metamorphosis-005.jpg', 'images/Orders-with-Complete-Metamorphosis-006.jpg', 'images/Orders-with-Complete-Metamorphosis-007.jpg', 'images/Orders-with-Complete-Metamorphosis-008.jpg', 'images/Orders-with-Complete-Metamorphosis-009.jpg', 'images/Orders-with-Complete-Metamorphosis-010.jpg', 'images/Orders-with-Complete-Metamorphosis-011.jpg', 'images/Orders-with-Complete-Metamorphosis-012.jpg', 'images/Orders-with-Complete-Metamorphosis-013.jpg', 'images/Orders-with-Complete-Metamorphosis-014.jpg', 'images/Orders-with-Complete-Metamorphosis-015.jpg', 'images/Orders-with-Complete-Metamorphosis-016.jpg', 'images/Orders-with-Complete-Metamorphosis-017.jpg', 'images/Orders-with-Complete-Metamorphosis-01